Data Preprocessing



In [ ]:
import pandas as pd
import requests 
import json
from datetime import datetime , timedelta
key = "biXmYSG60H0IuxAdhaZbvGPNDTBPwzoX"
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/'Dossier partagé avec Iheb BELGACEM'/BCG_Hackathon

/content/drive/.shortcut-targets-by-id/1yXfewQDE3qy07N5B0XsOVSfqgaaXT3R7/Dossier partagé avec Iheb BELGACEM/BCG_Hackathon


In [ ]:
file_name = 'datasets/washington.csv'
save_name = 'datasets/wash1.csv'
amont = "Av_Champs_Elysees-Washington"
aval = "Av_Champs_Elysees-Berri"

Date Features

In [ ]:
df = pd.read_csv(file_name, sep = ";")
df = df[["Date et heure de comptage","Débit horaire","Taux d'occupation","Etat trafic","Libelle noeud amont","Libelle noeud aval"]]
def filter(df,amont,aval):
    df = df[df["Libelle noeud amont"]==amont]
    df = df[df["Libelle noeud aval"]==aval]
    return df

def set_date(df):
    df["Date et heure de comptage"] = pd.to_datetime(df["Date et heure de comptage"],format='%Y-%m-%dT%H:%M:%S%z')
    return df
df = filter(df,amont,aval)
df = set_date(df)
df = df.sort_values(by = "Date et heure de comptage")
df = df.interpolate() #remove Nan by interpolation
df = df.reset_index()
df["date"] = [d.date() for d in df["Date et heure de comptage"]]
df["weekday"] = [d.weekday() for d in df["Date et heure de comptage"]]
df["weekofyear"] = [d.weekofyear for d in df["Date et heure de comptage"]]
df["month"] = [d.month for d in df["Date et heure de comptage"]]
df["year"] = [d.year for d in df["Date et heure de comptage"]]
df["time"] = [d.time() for d in df["Date et heure de comptage"]]

Weather Data (Hourly and Daily)

In [ ]:
#  ADD HOURLY DATA
count = 0
test = True
start = df.loc[0,"date"]
fin = df.loc[df.index[-1],"date"]
hd = []
while test : 
  print(count)
  url = "https://api.meteostat.net/v2/stations/hourly?station=07157&start="+start.strftime('%Y-%m-%d')+"&end="+(start+timedelta(9)).strftime('%Y-%m-%d')
  r = requests.get(url,headers={'x-api-key':key})
  d = json.loads(r.text)
  if len(d['data']) <10 : 
    print("Problem")
  else :
    hd = hd + d['data']
  start = start + timedelta(9)
  if start > fin : 
    test = False
  count = count +1
  time.sleep(3)
hd = pd.DataFrame.from_records(hd)
hd = hd[["time","temp","dwpt","rhum"]]
hd["time"] = pd.to_datetime(hd["time"],format='%Y-%m-%d %H:%M:%S')
hd = hd.sort_values(by = "time")
hd = hd.drop_duplicates().reset_index(drop = True)
hd = hd.interpolate()
hd = hd.rename(columns = {"time":"Date et heure de comptage"})
hd["date"] = [d.date() for d in hd["Date et heure de comptage"]]
hd["time"] = [d.time() for d in hd["Date et heure de comptage"]]
del hd['Date et heure de comptage']
df = pd.merge(df,hd,on =['time','date'], how = 'left').sort_values(by = "Date et heure de comptage")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


In [ ]:
# DAILY DATA
count = 0
test = True
start = df.loc[0,"date"]
fin = df.loc[df.index[-1],"date"]
dd = []
while test : 
  print(count)
  url = "https://api.meteostat.net/v2/stations/daily?station=07157&start="+start.strftime('%Y-%m-%d')+"&end="+min(start+timedelta(369),fin).strftime('%Y-%m-%d')
  r = requests.get(url,headers={'x-api-key':key})
  d = json.loads(r.text)
  if len(d['data']) <10 : 
    print("Problem")
  else :
    dd = dd + d['data']
  start = start + timedelta(369)
  if start > fin : 
    test = False
  count = count +1
  time.sleep(3)
dd = pd.DataFrame.from_records(dd)
dd["date"] = pd.to_datetime(dd["date"],format='%Y-%m-%d')
dd = dd.sort_values(by = "date")
dd = dd[["date","tavg","tmin","tmax","pres"]]
dd = dd.interpolate()
dd.date = dd.date.apply(lambda x : x.date())
df = pd.merge(df,dd,on ='date', how = 'left').sort_values(by = "Date et heure de comptage")

0
1


In [ ]:
df["mean_taux_occupation_past_week"] = df["Taux d'occupation"].rolling(24*7).mean().shift(24*6)
df["taux_occupation_past_week"] = df["Taux d'occupation"].shift(24*7)
df["mean_debit_horaire_past_week"] = df["Débit horaire"].rolling(24*7).mean().shift(24*6)
df["debit_horaire_past_week"] = df["Débit horaire"].shift(24*7)

In [ ]:
covid =pd.read_csv('datasets/covid.csv')
del covid['Unnamed: 0']
covid.date = pd.to_datetime(covid["date"],format='%Y-%m-%d')
covid.date = covid.date.apply(lambda x : x.date())
df = pd.merge(df,covid,on = 'date',how = 'left')
f = df[df['taux_occupation_sae'].isna()].index[-1]
df.loc[:f+1,'taux_occupation_sae'] = df.loc[f+1,'taux_occupation_sae']
# Mets à jours RETAIL and Taux d'occupation des lits de réa


retail =pd.read_csv('datasets/retail.csv')
del retail['Unnamed: 0']
retail.date = pd.to_datetime(retail["date"],format='%Y-%m-%d')
retail.date = retail.date.apply(lambda x : x.date())
df = pd.merge(df,retail,on = 'date',how = 'left') 
df['retail_and_recreation_percent_change_from_baseline'] = df['retail_and_recreation_percent_change_from_baseline'].interpolate()
df['retail_and_recreation_percent_change_from_baseline'] = df['retail_and_recreation_percent_change_from_baseline'].fillna(method = 'backfill')

confi = pd.read_csv('datasets/confinement couvre feu.csv',sep = ";")
confi = pd.concat([pd.get_dummies(confi.confinement, prefix= 'confin'),confi], axis= 1).drop(columns = ['confinement'])
confi.date = pd.to_datetime(confi["date"],format='%Y-%m-%d')
confi.date = confi.date.apply(lambda x : x.date())
df = pd.merge(df,confi,on = 'date',how = 'left') 

jf = pd.read_csv('datasets/feries.csv')
jf.nom_jour_ferie = 1
jf = jf.drop(['annee','zone'],axis = 1)
jf.date = pd.to_datetime(jf["date"],format='%Y-%m-%d')
jf.date = jf.date.apply(lambda x : x.date())
jf.rename(columns = {"nom_jour_ferie":"ferie"}, inplace = True)
df = pd.merge(df,jf,on = 'date',how = 'left') 
df.ferie = df.ferie.fillna(0)
df.ferie = df.ferie.apply(lambda x : int(x))
df=df[24*7+24*6:].reset_index(drop = True)

In [ ]:
df.to_csv(save_name)

In [ ]:
df

,index,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic,Libelle noeud amont,Libelle noeud aval,date,weekday,weekofyear,month,year,time,temp,dwpt,rhum,tavg,tmin,tmax,pres,mean_taux_occupation_past_week,taux_occupation_past_week,mean_debit_horaire_past_week,debit_horaire_past_week,taux_occupation_sae,retail_and_recreation_percent_change_from_baseline,confin_0,confin_1,confin_2,couvrefeu,ferie
0,6965,2019-11-14 04:00:00+01:00,295.0,3.20167,Fluide,Av_Champs_Elysees-Washington,Av_Champs_Elysees-Berri,2019-11-14,3,46,11,2019,04:00:00,5.1,4.2,94.0,5.6,4.6,6.5,997.9,15.835636,5.89833,837.223214,298.0,12.289109,96.0,1,0,0,0,0
1,7452,2019-11-14 05:00:00+01:00,282.0,3.27389,Fluide,Av_Champs_Elysees-Washington,Av_Champs_Elysees-Berri,2019-11-14,3,46,11,2019,05:00:00,5.3,4.3,93.0,5.6,4.6,6.5,997.9,15.809154,2.29278,835.616071,260.0,12.289109,96.0,1,0,0,0,0
2,6966,2019-11-14 06:00:00+01:00,450.0,5.28222,Fluide,Av_Champs_Elysees-Washington,Av_Champs_Elysees-Berri,2019-11-14,3,46,11,2019,06:00:00,5.5,3.3,86.0,5.6,4.6,6.5,997.9,15.828470,6.15222,835.044643,430.0,12.289109,96.0,1,0,0,0,0
3,6967,2019-11-14 07:00:00+01:00,793.0,9.07722,Fluide,Av_Champs_Elysees-Washington,Av_Champs_Elysees-Berri,2019-11-14,3,46,11,2019,07:00:00,5.2,3.4,88.0,5.6,4.6,6.5,997.9,15.854842,7.03278,836.508929,692.0,12.289109,96.0,1,0,0,0,0
4,6968,2019-11-14 08:00:00+01:00,1018.0,15.51111,Pré-saturé,Av_Champs_Elysees-Washington,Av_Champs_Elysees-Berri,2019-11-14,3,46,11,2019,08:00:00,5.2,3.5,89.0,5.6,4.6,6.5,997.9,15.900355,12.87889,839.622024,1128.0,12.289109,96.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9331,7547,2020-11-29 20:00:00+01:00,859.0,13.09667,Fluide,Av_Champs_Elysees-Washington,Av_Champs_Elysees-Berri,2020-11-29,6,48,11,2020,20:00:00,1.0,-0.9,87.0,2.5,1.0,6.0,1023.0,5.704904,9.45389,484.666667,500.0,72.933663,34.0,0,0,1,0,0
9332,7449,2020-11-29 21:00:00+01:00,568.0,7.50778,Fluide,Av_Champs_Elysees-Washington,Av_Champs_Elysees-Berri,2020-11-29,6,48,11,2020,21:00:00,1.0,-0.9,87.0,2.5,1.0,6.0,1023.0,5.720698,4.75833,485.791667,359.0,72.933663,34.0,0,0,1,0,0
9333,7548,2020-11-29 22:00:00+01:00,391.0,4.38000,Fluide,Av_Champs_Elysees-Washington,Av_Champs_Elysees-Berri,2020-11-29,6,48,11,2020,22:00:00,2.0,0.1,87.0,2.5,1.0,6.0,1023.0,5.733952,2.91389,486.714286,256.0,72.933663,34.0,0,0,1,0,0
9334,7549,2020-11-29 23:00:00+01:00,218.0,2.11500,Fluide,Av_Champs_Elysees-Washington,Av_Champs_Elysees-Berri,2020-11-29,6,48,11,2020,23:00:00,2.0,0.1,87.0,2.5,1.0,6.0,1023.0,5.743793,1.89722,487.482143,149.0,72.933663,34.0,0,0,1,0,0
